In [ ]:
!pip install "pyautogen[retrievechat]==0.2.6" -qqq

# **OpenAI API 키 설정**

In [ ]:
import json

openai_api_key = "자신의 API 키 입력"

with open('OAI_CONFIG_LIST.json', 'w') as f:
  config_list = [
      {
          "model": "gpt-4-turbo-preview",
          "api_key": openai_api_key
      },
      {
          "model": "gpt-4o",
          "api_key": openai_api_key,
      },
      {
          "model": "dall-e-3",
          "api_key": openai_api_key,
      }
  ]

  json.dump(config_list, f)

# **에이전트에 사용할 설정 불러오기**

In [ ]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    file_location=".",
    # 특정 모델만 사용하도록 필터링
    filter_dict={
        "model": {"gpt-4-turbo-preview"},
    },
)

llm_config = {
    "config_list": config_list,
    "temperature": 0,
}

# **AutoGen의 핵심 구성요소인 UserProxyAgent와 AssistantAgent**

In [ ]:
from autogen import AssistantAgent, UserProxyAgent

# AI 모델을 실행하는 역할
# AssistantAgent("에이전트 이름", AI 모델 설정)
assistant = AssistantAgent("assistant", llm_config=llm_config)
# 사용자를 대신해 명령을 내리는 에이전트
# 터미널 명령을 실행하거나 AI가 생성한 코드를 실행할 수도 있다
user_proxy = UserProxyAgent("user_proxy", # 에이전트 이름
                            # 종료 조건 설정
                            # 현재 조건은 content 값이 있는지 확인하고
                            # rstrip() -> 오른쪽 공백 제거
                            # 문자열이 TERMINATE로 끝나면 종료
                            is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
                            # 사용자가 직접 입력할 수 없도록 설정
                            # "ALWAYS"로 설정하면 AI가 사용자에게 입력을 요청할 수도 있음
                            human_input_mode="NEVER",
                            # "work_dir": "coding" -> 코드를 실행할 작업 폴더
                            # "user_docker": False 도커환경에서 실행하지 않도록 설정
                            code_execution_config={"work_dir": "coding", "user_docker": False})

# **삼성전자의 3개월 주식 가격 그래프를 그리는 작업 실행**

In [ ]:
# 에이전트와 대화를 시작
user_proxy.initiate_chat(assistant, message="""
삼성전자의 지난 3개월 주식 가격 그래프를 그려서 samsung_stock_price.png 파일로 저장해줘.
plotly 라이브러리를 사용하고 그래프 아래를 투명한 녹색으로 채워줘.
값을 잘 확인할 수 있도록 y축은 구간 최소값에서 시작하도록 해줘.
이미지 비율은 보기 좋게 적절히 설정해줘.
""")

# **RAG 에이전트 클래스를 사용한 작업 실행**

In [ ]:
import autogen
# 검색(Retrieval) 기능을 갖춘 도우미 AI 에이전트
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
# 검색(Retrieval) 기능이 있는 사용자 프록시 에이전트
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

assistant = RetrieveAssistantAgent(
    name="assistant",
    # 기본적인 역할을 정의
    system_message="You are a helpful assistant,",
    llm_config=llm_config,
)

# 사용자의 질문을 받아 외부 문서에서 관련 정보를 검색해주는 역할
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    retrieve_config={
        # 	질의응답(Question-Answering) 태스크 수행
        "task": "qa",
        # AutoGen 공식 문서(README)를 검색 대상 문서로 설정
        "docs_path": "https://raw.githubusercontent.com/microsoft/autogen/main/README.MD",
        # 문서 검색에 사용할 임베딩 모델 지정
        "collection_name": "default-sentence-transformers"
    },
)

# 도우미 AI의 상태를 초기화
# 과거의 대화 내용을 삭제하고 새롭게 시작할 준비
assistant.reset()
ragproxyagent.initiate_chat(assistant, problem="AutoGen이 뭐야?")

# assistant (to ragproxyagent):
# AutoGen은 여러 에이전트가 상호 대화하여 작업을 해결할 수 있는 LLM(Large Language Model) 애플리케이션 개발을 가능하게 하는 프레임워크입니다. AutoGen 에이전트는 사용자 정의 가능하며, 대화 가능하고, 인간 참여를 원활하게 허용합니다. LLM, 인간 입력, 도구의 조합을 사용하는 다양한 모드에서 작동할 수 있습니다.

# **외부 정보를 활용하지 못하는 기본 에이전트의 답변**

In [ ]:
assistant.reset()
userproxyagent = autogen.UserProxyAgent(
    name="userproxyagent",
)
userproxyagent.initiate_chat(assistant, message="Autogen이 뭐야?")

# assistant (to userproxyagent):
# "Autogen"은 자동 생성을 의미하는 용어로, 주로 컴퓨터 프로그래밍에서 사용됩니다. 이는 코드, 문서, 또는 다른 데이터를 자동으로 생성하는 프로세스를 가리킵니다. 이는 반복적인 작업을 줄이고, 효율성을 높이며, 오류를 줄일 수 있습니다. 특정 컨텍스트에 따라 "Autogen"의 정확한 의미는 다를 수 있습니다.

# **OpenAI 임베딩 모델을 사용하도록 설정하기**

In [ ]:
# 임베딩 함수를 불러오는 모듈
from chromadb.utils import embedding_functions

# OpenAI의 텍스트 임베딩 모델을 사용하여 문서를 벡터화
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=openai_api_key,
    model_name="text-embedding-3-small"
)

ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    retrieve_config={
        "task": "qa",
        "docs_path": "https://raw.githubusercontent.com/microsoft/autogen/main/README.MD",
        # 임베딩 모델 지정
        "embedding_function": openai_ef,
        "collection_name": "openai-embedding-3",
    },
)

assistant.reset()
ragproxyagent.initiate_chat(assistant, problem="Autogen이 뭐야?")

# assistant (to ragproxyagent):
# AutoGen은 여러 에이전트가 상호 대화하여 작업을 해결할 수 있는 LLM(Large Language Model) 애플리케이션 개발을 가능하게 하는 프레임워크입니다. AutoGen 에이전트는 사용자 정의 가능하며, 대화 가능하고, 인간 참여를 원활하게 허용합니다. LLM, 인간 입력, 도구의 조합을 사용하는 다양한 모드에서 작동할 수 있습니다.

# **대화에 참여할 에이전트**

In [ ]:
# 종료 메시지 함수
def termination_msg(x):
  return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

# RAG를 사용하지 않는 사용자 역할 에이전트
# 질문을 던지는 관리자 역할
user = autogen.UserProxyAgent(
    name="Admin",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    # 역할 정의: 질문을 던지고 작업을 지시하는 관리자
    system_message="The boss who ask questions and give tasks",
    # 코드 실행 기능 없음
    code_execution_config=False,
    # 자동 응답 설정
    default_auto_reply="Reply `TERMINATE` if the task is done.",
)

# RAG를 사용하는 사용자 역할 에이전트
# 추가적인 문서 검색 기능을 가진 관리자
user_rag = RetrieveUserProxyAgent(
    name="Admin_RAG",
    is_termination_msg=termination_msg,
    system_message="Assistant who has extra content retrieval power for solving difficult problems.",
    human_input_mode="NEVER",
    # 최대 3번까지 자동 응답 허용
    max_consecutive_auto_reply=3,
    code_execution_config=False,
    retrieve_config={
        # 코드 관련 질문을 처리하는 역할
        "task": "code",
        "docs_path": "https://raw.githubusercontent.com/microsoft/autogen/main/samples/apps/autogen-studio/README.MD",
        # 한 번에 1000개의 토큰씩 문서를 나눠서 검색
        "chunk_token_size": 1000,
        "collection_name": "groupchat-rag",
    }
)

# 프로그래머 역할의 에이전트
coder = AssistantAgent(
    name="Senior_Python_Engineer",
    is_termination_msg=termination_msg,
    system_message="You are a senior python engineer. Reply `TERMINATE` in the end when everything is done,",
    llm_config=llm_config,
)

# 프로덕트 매니저 역할의 에이전트
pm = autogen.AssistantAgent(
    name="Product_Manager",
    is_termination_msg=termination_msg,
    system_message="You are a product manager. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config
)

PROBLEM = "AutoGen Studio는 무엇이고 AutoGen Studio로 어떤 제품을 만들 수 있을까?"

# **RAG 사용 여부에 따른 2개의 그룹챗 정의 및 실행**

In [ ]:
# 에이전트 초기화 메서드
# 에이전트가 이전 대화를 기억하기 때문
def _reset_agents():
  user.reset()
  user_rag.reset()
  coder.reset()
  pm.reset()

def rag_chat():
  _reset_agents()

  # 그룹 챗 생성
  groupchat = autogen.GroupChat(
      # 참여 에이전트
      agents=[user_rag, coder, pm],
      # 초기 메시지 없음
      messages=[],
      # 최대 12번 대화 진행 가능
      max_round=12,
      # 순서대로 돌아가면서 발언
      speaker_selection_method="round_robin"
  )
  # 그룹 챗을 관리하는 매니저 역할
  # 어떤 에이전트가 맡아서 하는게 아닌 그냥 컨트롤러 역활?
  manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

  # PROBLEM을 던져서 대화를 시작
  user_reg.initiate_chat(
      manager,
      problem=PROBLEM,
  )

def norag_chat():
  _reset_agents()

  groupchat = autogen.GroupChat(
      agents=[user, coder, pm],
      messages=[],
      max_round=12,
      # 자동으로 적절한 에이전트가 발언
      speaker_selection_method="auto",
      # 같은 에이전트가 연속으로 발언하는 것을 방지
      allow_repeat_speaker=False
  )
  manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

  user.initiate_chat(
      manager,
      message=PROBLEM,
  )

# **2개의 그룹챗을 실행한 결과 비교**

In [ ]:
norag_chat()

# AutoGen Studio는 자동화된 코드 생성 도구입니다. 이 도구를 사용하면 개발자들이 더 빠르게, 더 효율적으로 코드를 작성할 수 있습니다.
# AutoGen Studio를 사용하면 다양한 유형의 소프트웨어 제품을 만들 수 있습니다. 예를 들어, 웹 애플리케이션, 모바일 애플리케이션, 데스크톱 애플리케이션, API, 데이터베이스 등을 만들 수 있습니다.
# ...

In [ ]:
rag_chat()

# AutoGen Studio는 AutoGen 프레임워크를 기반으로 한 AI 앱입니다. 이 앱은 AI 에이전트를 빠르게 프로토타입화하고, 스킬을 향상시키고, 워크플로우로 구성하고, 작업을 완료하기 위해 그들과 상호 작용하는 데 도움을 줍니다. 이 앱은 GitHub의 [microsoft/autogen](https://github.com/microsoft/autogen/tree/main/samples/apps/autogen-studio)에서 코드를 찾을 수 있습니다.
# AutoGen Studio를 사용하면 다음과 같은 기능을 수행할 수 있습니다:
# - 에이전트를 구축/구성하고, 그들의 구성(예: 스킬, 온도, 모델, 에이전트 시스템 메시지, 모델 등)을 수정하고, 워크플로우로 구성합니다.
# ...

# **실습 준비하기**

In [ ]:
import os
import re
import time
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import matplotlib.pyplot as plt
import PIL
import requests
from openai import OpenAI
from PIL import Image

from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.img_utils import _to_pil, get_image_data
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent

config_list_4o = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    filter_dict={
        "model": ["gpt-4o"],
    },
)

config_list_dalle = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    filter_dict={
        "model": ["dall-e-3"],
    },
)

# **DALLEAgent 정의**

In [ ]:
# 이미지 생성 함수
# -> str = 타입 힌트 -> 반환값 str
def delle_call(client, prompt, model="dall-e-3", size="1024x1024", quality="standard", n=1) -> str:
  # OpenAI의 DALL-E 3 모델을 사용해 이미지 생성
  response = client.images.generate(
      model=model,
      prompt=prompt,
      size=size,
      quality=quality,
      n=n,
  )

  image_url = response.data[0].url
  img_data = get_image_data(image_url)

  return img_data

# 대화형 에이전트 클래스
# ConversableAgent 상속
class DALLEAgent(ConversableAgent):
  # **kwargs -> 키워드 인수
  # **kwargs는 함수에 이름이 지정된 인수들을 딕셔너리 형태로 받을 때 사용
  def __init__(self, name, llm_config, **kwargs):
    # 부모 클래스 초기화
    super().__init__(name, llm_config=llm_config, **kwargs)

    try:
      # 키 가져오기
      config_list = llm_config["config_list"]
      api_key = config_list[0]["api_key"]
    except Exception as e:
      print("Unable to fetch API Key, because", e)
      # 없으면 환경변수에서 찾음
      api_key = os.getenv("OPENAI_API_KEY")
    # OpenAI API를 사용하기 위한 클라이언트 초기화
    self.client = OpenAI(api_key=api_key)

    # register_reply
    # 에이전트가 응답할 방식을 설정하는 메서드
    # 어떤 메시지를 받으면 어떻게 응답할지 등록하는 역할

    # [Agent, None]
    # 메시지를 받을 수 있는 에이전트의 유형을 지정하는 부분
    # Agent: 메시지를 받는 에이전트가 Agent 유형이어야 한다는 것을 의미
    # None: None은 이 부분에서 메시지를 받는 주체에 제한을 두지 않겠다는 의미
    # 특정 조건이나 에이전트 유형에 구애받지 않고 메시지를 받을 수 있게 설정

    # DALLEAgent.generate_dalle_reply
    # 메시지를 처리할 함수를 지정
    self.register_reply([Agent, None], DALLEAgent.generate_dalle_reply)

  # 사용자에게 이미지를 생성하여 반환하는 함수
  def generate_dalle_reply(self, messages, sender, config):
    #  만약 config가 제공되지 않으면 self.client (OpenAI 클라이언트)를 사용
    client = self.client if config is None else config

    if client is None:
      # False -> 요청을 처리할 수 없음을 나타내는 값
      # None -> 이미지 데이터를 반환할 수 없음을 나타냄(에러)
      return False, None
    if messages is None:
      # messages가 None이라면 대신에 self._oai_messages[sender]를 사용하여 해당 발신자의 메시지 목록을 가져온다
      # messages가 None인 경우에는 클래스 내부에 저장된 발신자에 대한 메시지를 사용하여 처리를 이어가겠다는 의미
      messages = self._oai_messages[sender]

    # -1 -> 리스트의 맨 마지막 요소 가져옴
    # 가장 최근에 전달된 메시지에서 이미지를 생성할 프롬프트 추출
    prompt = messages[-1]["content"]

    # 이미지 요청
    img_data = dalle_call(client=self.client, prompt=prompt)

    # 이미지 출력
    plt.imshow(_to_pil(image_data))
    plt.axis("off")
    plt.show()

    # 응답으로 이미지가 성공적으로 생성
    return True, 'result.jpg'

# **이미지 생성 에이전트 실행**

In [ ]:
painter = DALLEAgent(name="Painter", llm_config={"config_list": config_list_dalle})

user_proxy = UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=0
)

# 이미지 생성 작업 실행하기
user_proxy.initiate_chat(
    painter,
    message="갈색의 털을 가진 귀여운 강아지를 그려줘",
)